<a href="https://colab.research.google.com/github/Learnheart/URL-Classification/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import tree
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier


!pip3 install pyvi
from pyvi import ViUtils



In [ ]:
import sklearn
print(sklearn.__version__)

1.4.1.post1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
names=['URL','Category']
df = pd.read_csv('/content/drive/MyDrive/NCKH/Data/full training data.csv',names=names, usecols=[0,2],na_filter=False, encoding='latin-1')
print(df.shape)
df[10632:10640]

(379119, 2)


,URL,Category
10632,bonbanh.com/ha-noi/oto/suzuki-grand_vitara-nam...,Automotive
10633,bonbanh.com/oto/mitsubishi-mirage-so-san-cu-da...,Automotive
10634,"mitsubishibacquang.bonbanh.com/chi-tiet-xe/id,...",Automotive
10635,kienthuc.net.vn/o-to-xe-may/nhung-quy-dinh-moi...,Automotive
10636,bonbanh.com/ha-noi/oto/mitsubishi-sf000000020/...,Automotive
10637,bonbanh.com/xe-porsche-cayenne--2008-3751103,Automotive
10638,otofun.net/threads/hoi-kinh-nghiem-dung-bong-d...,Automotive
10639,bonbanh.com/oto/toyota-corolla_altis-sedan-4-c...,Automotive


In [ ]:
import re
from sklearn.model_selection import train_test_split

def url_to_text(url):
    # remove stopwords
    url = url.replace('.html','').replace('.htm','').replace('http://','').replace('https://','')
    url = re.sub('^(.*?/)','/', url) # remove domains
    url = re.sub('[0-9]+', '', url)
    url = re.sub('[_\-/]+', ' ', url)

    return ViUtils.remove_accents(url.lower()).decode()



In [ ]:
df['URL'] = df['URL'].apply(url_to_text)

In [ ]:
#Drop duplicate
df = df.drop_duplicates()
df

,URL,Category
0,can tho oto toyota vios nam,Automotive
1,"lam dong oto so tu dong tu nam page,",Automotive
2,"oto so san nam sf page,",Automotive
3,threads cong nghe rua xe may o ha noi.,Automotive
4,oto kia nhap khau so tu dong mau trang cu da ...,Automotive
...,...,...
379114,hanh vi xa rac noi cong cong tang che tai de ...,Environment
379115,vinh phuc lay y kien danh gia tac dong moi tr...,Environment
379116,tim ra nguyen nhan khien mat nuoc song be tro...,Environment
379117,bac bo va bac trung bo chuan bi don dot ret m...,Environment


In [ ]:

#Your existing code
X = df['URL'].apply(url_to_text)
y = df['Category']

# Split the data into train, validation, and test sets
train, test = train_test_split(df, random_state=33, test_size=0.1)
train, val = train_test_split(train, random_state=44, test_size=0.2)

# Assuming 'target_column' is the target column in your DataFrame
X_train, y_train = train['URL'].apply(url_to_text), train['Category']
X_val, y_val = val['URL'].apply(url_to_text), val['Category']
X_test, y_test = test['URL'].apply(url_to_text), test['Category']

#One hot processing

#Vectorizer

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import classification_report, confusion_matrix

# # Initialize the TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()

# # Fit and transform the training data
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# # Transform the validation and test data
# X_val_tfidf = tfidf_vectorizer.transform(X_val)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
vectorizer = TfidfVectorizer() #transforms text data into numerical feature vectors
pipeline = Pipeline([ #pipeline contain all compponents as tuple
    ('vect', vectorizer),
    ('chi',  SelectKBest(chi2, k=1200)), #uses the chi-squared (chi2) statistical test to select the k=1200 best features.
    ('clf', DecisionTreeClassifier())
])

In [ ]:

model = pipeline.fit(X_train, y_train)

y_test_pred = model.predict(X_test)

print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))


                        precision    recall  f1-score   support

            Automotive       0.98      0.96      0.97      3108
    Books & Literature       0.87      0.97      0.92      1695
    Business & Finance       0.98      0.98      0.98      2887
               Careers       0.97      0.94      0.95      1415
             Education       0.98      0.98      0.98      1772
   Entertainment & Art       0.95      0.95      0.95      4593
           Environment       0.76      0.75      0.76      1013
Family & Relationships       0.89      0.88      0.89      1530
          Food & Drink       0.88      0.90      0.89      2943
                 Games       0.96      0.93      0.95       984
        Healthy Living       0.94      0.94      0.94      1407
         Home & Garden       0.97      0.97      0.97      1702
       Laws & Policies       0.87      0.86      0.86       995
       News & Politics       0.89      0.88      0.89      2203
           Real Estate       0.80      

In [ ]:
new_input_url = 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/'
new_input_processed = url_to_text(new_input_url)

model_prediction = model.predict([new_input_url])

print(f"Prediction for '{new_input_url}': {model_prediction}")


Prediction for 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/': ['Healthy Living']


In [ ]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/NCKH/HyperModel/decision_tree_1.4.1.pkl')

['/content/drive/MyDrive/NCKH/HyperModel/decision_tree_1.4.1.pkl']

#Origin model

In [ ]:
import time
# Record the start time
start_time = time.time()

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf, y_train)

# Record the end time
end_time = time.time()

# Calculate and print the training time
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

# Evaluate the model on the test set
y_test_pred = clf.predict(X_test_tfidf)

# Print the classification report and confusion matrix
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

Training time: 140.40500259399414 seconds
                        precision    recall  f1-score   support

            Automotive       0.95      0.98      0.96      3108
    Books & Literature       0.97      0.94      0.95      1695
    Business & Finance       0.98      0.98      0.98      2887
               Careers       0.97      0.95      0.96      1415
             Education       0.98      0.98      0.98      1772
   Entertainment & Art       0.95      0.94      0.95      4593
           Environment       0.76      0.76      0.76      1013
Family & Relationships       0.89      0.89      0.89      1530
          Food & Drink       0.89      0.91      0.90      2943
                 Games       0.97      0.96      0.97       984
        Healthy Living       0.95      0.93      0.94      1407
         Home & Garden       0.97      0.97      0.97      1702
       Laws & Policies       0.88      0.86      0.87       995
       News & Politics       0.89      0.89      0.89      22

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)

# Print accuracy
print("Accuracy:", accuracy)


Accuracy: 0.9366955053808821


In [ ]:
new_input_url = 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/'
new_input_processed = url_to_text(new_input_url)

# Make prediction for the new input
model_prediction = clf.predict(tfidf_vectorizer.transform([new_input_processed]))

# Print the prediction
print(f"Prediction for '{new_input_url}': {model_prediction}")


Prediction for 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/': ['Healthy Living']


In [ ]:
import joblib
joblib.dump(clf, '/content/drive/MyDrive/NCKH/HyperModel/decision_tree_origin.pkl')

['/content/drive/MyDrive/NCKH/HyperModel/decision_tree_origin.pkl']

In [ ]:
testing = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/decision_tree_origin.pkl')

model_test = testing.predict(tfidf_vectorizer.transform([new_input_processed]))

# Print the prediction
print(f"Prediction for '{new_input_url}': {model_test}")

Prediction for 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/': ['Healthy Living']


#Hyper param


In [ ]:
param_grid = {
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__criterion': ['gini', 'entropy'],
    'clf__class_weight': [None, 'balanced']
}


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import time
import logging

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, n_jobs=-1)

logging.basicConfig(level=logging.INFO)
logging.info("Grid search started...")

grid_search.fit(X_train, y_train)
logging.info("Grid search completed.")

# Accessing cross-validated scores
cv_results = grid_search.cv_results_
mean_test_scores = cv_results['mean_test_score']
params = cv_results['params']

# Logging progress
for mean_score, params in zip(mean_test_scores, params):
    logging.info("Mean Test Score: %f", mean_score)
    logging.info("Hyperparameters: %s", params)


for mean_score, params in zip(mean_test_scores, params):
    print("Mean test score: ", mean_score)
    print("Hyperparameter", params)
    print("-------------------")


Fitting 3 folds for each of 432 candidates, totalling 1296 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


Mean test score:  0.8435989590548129
Hyperparameter clf__class_weight
-------------------
Mean test score:  0.8499368165960806
Hyperparameter clf__criterion
-------------------
Mean test score:  0.8571465289065321
Hyperparameter clf__max_depth
-------------------
Mean test score:  0.8382722850993162
Hyperparameter clf__max_features
-------------------
Mean test score:  0.8511567454070096
Hyperparameter clf__min_samples_leaf
-------------------
Mean test score:  0.8442986717392253
Hyperparameter clf__min_samples_split
-------------------


In [ ]:
# Get the best parameters and best score
print("Best parameters found:")
print(grid_search.best_params_)
print("Best cross-validation score:")
print(grid_search.best_score_)

# Evaluate the best model on the test set
best_clf = grid_search.best_estimator_
y_test_pred = best_clf.predict(X_test)

# Print the classification report and confusion matrix
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

Best parameters found:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best cross-validation score:
0.9249265390295953
                        precision    recall  f1-score   support

            Automotive       0.95      0.98      0.97      3050
    Books & Literature       0.96      0.93      0.94      1160
    Business & Finance       0.98      0.98      0.98      2848
               Careers       0.89      0.88      0.88       650
             Education       0.98      0.98      0.98      1800
   Entertainment & Art       0.94      0.94      0.94      4555
           Environment       0.68      0.74      0.71       922
Family & Relationships       0.89      0.89      0.89      1558
          Food & Drink       0.89      0.90      0.90      2940
                 Games       0.91      0.88      0.90       341
        Healthy Living       0.95      0.94      0.94      1452
         Home & Garden       0.97      0.97      0.97      1665
       Laws & Policies       

In [ ]:
import joblib

# Save the best model to a file
joblib.dump(grid_search, '/content/drive/MyDrive/NCKH/HyperModel/decisiontree1.pkl')



['/content/drive/MyDrive/NCKH/HyperModel/decisiontree1.pkl']

In [ ]:
# Later, you can load the model from the file
model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/decisiontree1.pkl')
new_input_url = 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/'
new_input_processed = url_to_text(new_input_url)

# Make prediction for the new input
model_prediction = model.predict([new_input_url])

# Print the prediction
print(f"Prediction for '{new_input_url}': {model_prediction}")


Prediction for 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/': ['Environment']
